In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ct

# --- Parámetros del controlador PI ---
Kp = 0.5
Ti = 1.0
C = Kp * (1 + ct.tf([1], [Ti, 0]))  # PI(s) = Kp*(1 + 1/(Ti*s))

# --- Planta lineal ---
s = ct.tf('s')
G = 1 / (s + 1)**3

# --- Característica de la válvula (no lineal) ---
def valve(v):
    return v**4

# --- Compensación por linealización aproximada ---
def inv_valve_piecewise(v):
    """Función inversa aproximada (lineal a trozos)"""
    u = np.zeros_like(v)
    v = np.asarray(v)
    u = np.where((v > 0) & (v <= 3), 0.433 * v, u)
    u = np.where((v > 3) & (v < 16), 0.0538 * v + 1.139, u)
    return u

# --- Simulación en lazo cerrado no lineal ---
def simulate(reference_change, compensated=False):
    """Simula el sistema con o sin compensación"""
    t = np.linspace(0, 50, 2000)
    r = np.ones_like(t) * reference_change[0]
    r[t > 10] = reference_change[1]  # salto en la referencia

    y = np.zeros_like(t)
    u = np.zeros_like(t)
    v = np.zeros_like(t)
    e_int = 0.0

    dt = t[1] - t[0]
    sys = ct.ssdata(G)  # modelo en espacio de estados
    A, B, Cmat, D = sys.A, sys.B, sys.C, sys.D
    x = np.zeros(A.shape[0])

    for k in range(1, len(t)):
        e = r[k] - y[k-1]
        e_int += e * dt
        v[k] = Kp * e + (Kp/Ti) * e_int   # salida del PI

        if compensated:
            u[k] = inv_valve_piecewise(v[k])
        else:
            u[k] = v[k]

        # salida de la válvula
        q = valve(u[k])

        # integración del sistema
        dx = A @ x + B.flatten() * q
        x = x + dx * dt
        y[k] = Cmat @ x + D * q

    return t, r, y, u, v

# --- Escenarios ---
scenarios = [(0.2, 0.3), (1.0, 1.1), (5.0, 5.1)]
fig, axes = plt.subplots(3, 2, figsize=(10, 9), sharex=True)

for i, (v1, v2) in enumerate(scenarios):
    # Sin compensación
    t, r, y, u, v = simulate((v1, v2), compensated=False)
    axes[i, 0].plot(t, r, 'k--', label='Referencia')
    axes[i, 0].plot(t, y, 'b', label='Salida')
    axes[i, 0].set_title(f"Sin compensación (de {v1} a {v2})")
    axes[i, 0].grid(True)
    axes[i, 0].legend()

    # Con compensación
    t, r, y, u, v = simulate((v1, v2), compensated=True)
    axes[i, 1].plot(t, r, 'k--', label='Referencia')
    axes[i, 1].plot(t, y, 'r', label='Salida')
    axes[i, 1].set_title(f"Con compensación (de {v1} a {v2})")
    axes[i, 1].grid(True)
    axes[i, 1].legend()

for ax in axes[-1, :]:
    ax.set_xlabel("Tiempo [s]")
for ax in axes[:, 0]:
    ax.set_ylabel("Salida")

plt.tight_layout()
plt.show()
